## Initial analyses - IPUMS data only, no weights accounted for

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format='retina'

### Loading Data

In [3]:
ipums = pd.read_csv('data/IPUMS_2019.csv')

In [4]:
ipums_titles = pd.read_csv('data/ind_indnaics_crosswalk_2000_onward_without_code_descriptions.csv')

In [5]:
ipums_titles = ipums_titles.iloc[2:]

In [6]:
ipums_titles = ipums_titles.iloc[:,10:]

In [7]:
county_info = pd.read_csv('data/county_to_regions_key - Sheet1.csv')

### Cleaning

In [8]:
ipums = ipums[['STATEFIP', 
               'COUNTYFIP',
              'CITY',
              'INDNAICS',
              'INCTOT',
              'INCWAGE']]

In [9]:
ca_ipums = ipums.loc[ipums['STATEFIP'] == 6].copy()

In [10]:
ca_ipums = ca_ipums.reset_index()

In [11]:
len(ca_ipums)

380091

In [12]:
def normalize_titles(col):
    col = col.astype(str)
    col = col.str.strip()
    col = col.str.lower()
    col = col.str.replace('&', 'and')
    col = col.apply(lambda x:''.join([i for i in x if i not in string.punctuation]))
    return col

In [13]:
ca_ipums['INDNAICS'] = normalize_titles(ca_ipums['INDNAICS'])

In [14]:
ipums_titles['2018 Onward ACS/PRCS INDNAICS CODE'] = normalize_titles(ipums_titles['2018 Onward ACS/PRCS INDNAICS CODE'])

### Merging on NAICS Codes

In [15]:
ca_ipums.columns.values

array(['index', 'STATEFIP', 'COUNTYFIP', 'CITY', 'INDNAICS', 'INCTOT',
       'INCWAGE'], dtype=object)

In [16]:
ipums_titles.columns.values

array(['2018 Onward ACS/PRCS INDNAICS CODE', 'Industry Title'],
      dtype=object)

In [17]:
ipums_merged = pd.merge(ca_ipums, ipums_titles,
                              left_on = 'INDNAICS', right_on = '2018 Onward ACS/PRCS INDNAICS CODE')

In [18]:
print(len(ca_ipums), len(ipums_titles), ipums_merged.shape)

380091 402 (224526, 9)


In [19]:
ca_ipums[~ca_ipums['INDNAICS'].isin(ipums_titles['2018 Onward ACS/PRCS INDNAICS CODE'])]

,index,STATEFIP,COUNTYFIP,CITY,INDNAICS,INCTOT,INCWAGE
1,156867,6,73,0,0,0,0
3,156869,6,71,0,0,0,0
5,156871,6,37,3730,0,36700,0
7,156873,6,37,0,0,7400,0
8,156874,6,111,0,0,8500,0
...,...,...,...,...,...,...,...
380080,536946,6,97,0,0,9999999,999999
380082,536948,6,85,0,0,0,0
380085,536951,6,59,0,0,7000,0
380087,536953,6,59,0,0,0,0


In [20]:
ipums_merged

,index,STATEFIP,COUNTYFIP,CITY,INDNAICS,INCTOT,INCWAGE,2018 Onward ACS/PRCS INDNAICS CODE,Industry Title
0,156866,6,37,5140,4853,23100,23100,4853,Taxi and limousine service
1,157639,6,37,3730,4853,53500,28000,4853,Taxi and limousine service
2,157919,6,75,6290,4853,35800,1000,4853,Taxi and limousine service
3,158355,6,75,6290,4853,2800,1000,4853,Taxi and limousine service
4,158357,6,37,3730,4853,53500,28000,4853,Taxi and limousine service
...,...,...,...,...,...,...,...,...,...
224521,480571,6,97,0,3241m,80000,65000,3241m,Miscellaneous petroleum and coal products
224522,520559,6,73,0,3241m,9600,9600,3241m,Miscellaneous petroleum and coal products
224523,523405,6,37,0,3241m,53000,53000,3241m,Miscellaneous petroleum and coal products
224524,526138,6,85,0,3241m,98000,98000,3241m,Miscellaneous petroleum and coal products


In [21]:
print(f'''TOTAL ROWS: {len(ipums_merged)}''')
print('\n')
for column in ipums_merged.columns.values:
    total = len(ipums_merged)
    nulls = ipums_merged[column].isna().sum()
    percentage = (nulls / total) * 100
    print(f'''{column}: {nulls} null values, {percentage:.2f}%''')

TOTAL ROWS: 224526


index: 0 null values, 0.00%
STATEFIP: 0 null values, 0.00%
COUNTYFIP: 0 null values, 0.00%
CITY: 0 null values, 0.00%
INDNAICS: 0 null values, 0.00%
INCTOT: 0 null values, 0.00%
INCWAGE: 0 null values, 0.00%
2018 Onward ACS/PRCS INDNAICS CODE: 0 null values, 0.00%
Industry Title: 0 null values, 0.00%


In [22]:
for column in ipums_merged.columns.values:
    print(f'''{column}: {len(ipums_merged[column].unique())} unique values''')

index: 224526 unique values
STATEFIP: 1 unique values
COUNTYFIP: 35 unique values
CITY: 23 unique values
INDNAICS: 269 unique values
INCTOT: 7610 unique values
INCWAGE: 889 unique values
2018 Onward ACS/PRCS INDNAICS CODE: 269 unique values
Industry Title: 269 unique values


### Merging on counties

In [23]:
ipums_w_counties = pd.merge(ipums_merged, county_info, on='COUNTYFIP', how='left')

In [24]:
print(len(ipums_merged), len(county_info), ipums_w_counties.shape)

224526 58 (224526, 21)


In [25]:
ipums_w_counties.head()

,index,STATEFIP,COUNTYFIP,CITY,INDNAICS,INCTOT,INCWAGE,2018 Onward ACS/PRCS INDNAICS CODE,Industry Title,FIPS,...,State,"County, State",EDD County,Census County,Population - Households,Rural/Urban,Redstone Regions,WF Regions,CDI Regions,Population
0,156866,6,37,5140,4853,23100,23100,4853,Taxi and limousine service,6037.0,...,California,"Los Angeles, California",Los Angeles County,"Los Angeles County, California","2,207,265",Urban,Los Angeles,Greater Los Angeles,Los Angeles,"10,081,570"
1,157639,6,37,3730,4853,53500,28000,4853,Taxi and limousine service,6037.0,...,California,"Los Angeles, California",Los Angeles County,"Los Angeles County, California","2,207,265",Urban,Los Angeles,Greater Los Angeles,Los Angeles,"10,081,570"
2,157919,6,75,6290,4853,35800,1000,4853,Taxi and limousine service,6075.0,...,California,"San Francisco, California",San Francisco County,"San Francisco County, California","172,190",Urban,Bay Area,Bay Area,Bay Area,"874,961"
3,158355,6,75,6290,4853,2800,1000,4853,Taxi and limousine service,6075.0,...,California,"San Francisco, California",San Francisco County,"San Francisco County, California","172,190",Urban,Bay Area,Bay Area,Bay Area,"874,961"
4,158357,6,37,3730,4853,53500,28000,4853,Taxi and limousine service,6037.0,...,California,"Los Angeles, California",Los Angeles County,"Los Angeles County, California","2,207,265",Urban,Los Angeles,Greater Los Angeles,Los Angeles,"10,081,570"


In [26]:
ipums_w_counties_short = ipums_w_counties[['INDNAICS',
                                          'INCTOT',
                                          'INCWAGE',
                                          'Industry Title',
                                          'County',
                                          'CDI Regions',
                                          'Population']]

In [27]:
ipums_w_counties_short

,INDNAICS,INCTOT,INCWAGE,Industry Title,County,CDI Regions,Population
0,4853,23100,23100,Taxi and limousine service,Los Angeles,Los Angeles,"10,081,570"
1,4853,53500,28000,Taxi and limousine service,Los Angeles,Los Angeles,"10,081,570"
2,4853,35800,1000,Taxi and limousine service,San Francisco,Bay Area,"874,961"
3,4853,2800,1000,Taxi and limousine service,San Francisco,Bay Area,"874,961"
4,4853,53500,28000,Taxi and limousine service,Los Angeles,Los Angeles,"10,081,570"
...,...,...,...,...,...,...,...
224521,3241m,80000,65000,Miscellaneous petroleum and coal products,Sonoma,Bay Area,"499,772"
224522,3241m,9600,9600,Miscellaneous petroleum and coal products,San Diego,San Diego-Imperial,"3,316,073"
224523,3241m,53000,53000,Miscellaneous petroleum and coal products,Los Angeles,Los Angeles,"10,081,570"
224524,3241m,98000,98000,Miscellaneous petroleum and coal products,Santa Clara,Bay Area,"1,927,470"


### Statewide Industry % - wage > state mean

In [28]:
ca_median_income = ipums_w_counties_short['INCTOT'].median()
ca_median_income

40000.0

In [29]:
ca_median_wage = ipums_w_counties_short['INCWAGE'].median()
ca_median_wage

30000.0

In [30]:
ca_mean_income = ipums_w_counties_short['INCTOT'].mean()
ca_mean_income

64166.89874223921

In [31]:
ca_mean_wage = ipums_w_counties_short['INCWAGE'].mean()
ca_mean_wage

52498.9479614833

In [32]:
ipums_w_counties_short['Above CA Mean'] = ipums_w_counties_short['INCWAGE'] > ca_mean_wage

In [33]:
ipums_w_counties_short['Industry Counts'] = ipums_w_counties_short['Industry Title'].groupby(ipums_w_counties_short['Industry Title']).transform('count')

In [34]:
ipums_w_counties_short['CA High wage count'] = ipums_w_counties_short['Above CA Mean'].groupby(
    ipums_w_counties_short['Industry Title']).transform('sum')

In [35]:
ipums_w_counties_short['CA High wage percentage'] = (ipums_w_counties_short['CA High wage count'] / ipums_w_counties_short['Industry Counts']) * 100

In [36]:
high_wage_ca = ipums_w_counties_short.drop_duplicates(subset='Industry Title')
high_wage_ca = high_wage_ca[['Industry Title', 'Industry Counts', 'CA High wage count', 'CA High wage percentage']]
high_wage_ca = high_wage_ca.sort_values(by='CA High wage percentage', ascending=False)

In [37]:
high_wage_ca = high_wage_ca.reset_index().iloc[:,1:]

In [38]:
pd.set_option('display.max_rows', None)
high_wage_ca

,Industry Title,Industry Counts,CA High wage count,CA High wage percentage
0,Coal mining,3,3,100.000000
1,Pipeline transportation,16,14,87.500000
2,Software publishers,405,338,83.456790
3,Internet publishing and broadcasting and web s...,898,718,79.955457
4,"Engine, turbine, and power transmission equipm...",53,42,79.245283
5,"Electric and gas, and other combinations",364,287,78.846154
6,Not specified utilities,41,31,75.609756
7,Computer systems design and related services,6238,4617,74.014107
8,Computer and peripheral equipment manufacturing,524,383,73.091603
9,"Electric power generation, transmission and di...",558,401,71.863799


### Region Industry % - wage > region mean

In [39]:
ipums_w_counties_short['Region Mean Wage'] = ipums_w_counties_short['INCWAGE'].groupby(ipums_w_counties_short['CDI Regions']).transform('mean')

In [40]:
ipums_w_counties_short['Above Region Mean'] = ipums_w_counties_short['INCWAGE'] > ipums_w_counties_short['Region Mean Wage']

In [41]:
df = ipums_w_counties_short.groupby(['CDI Regions','Industry Title']).size().reset_index().rename(columns={0:'Region Industry Count'})

In [42]:
high_wage_regions = ipums_w_counties_short.groupby(['CDI Regions','Industry Title']).agg(
    {'Above Region Mean': 'sum'})
high_wage_regions['Region Industry Count'] = df['Region Industry Count'].values
high_wage_regions['Region High Wage Percentage'] = (
    high_wage_regions['Above Region Mean'] / high_wage_regions['Region Industry Count']) * 100

In [43]:
high_wage_regions = high_wage_regions.sort_values(['CDI Regions', 'Region High Wage Percentage'], ascending=(True, False))

In [44]:
pd.set_option('display.max_rows', None)
high_wage_regions

Above Region Mean  \
CDI Regions        Industry Title                                                          
Bay Area           Engine, turbine, and power transmission equipme...                  5   
                   Metal ore mining                                                    1   
                   Nonferrous metal, except aluminum, production a...                  1   
                   Tires                                                               1   
                   Veneer, plywood, and engineered wood products                       1   
                   Internet publishing and broadcasting and web se...                565   
                   Software publishers                                               184   
                   Agricultural implements                                             4   
                   Data processing, hosting, and related services                     99   
                   Computer systems design and related services                     2710   
                   Computer and peripheral equipment manufacturing                   255   
                   Pharmaceuticals and medicines                                     262   
                   Communications, audio, and video equipment                         86   
                   Miscellaneous petroleum and coal products                           5   
                   Petroleum refining                                                 58   
                   Management of companies and enterprises                            56   
                   Other information services, except libraries an...                 31   
                   Electric and gas, and other combinations                           65   
                   Telecommunications, except wired telecommunicat...                 80   
                   Electric power generation, transmission and dis...                 60   
                   Nondepository credit and related activities                       222   
                   Navigational, measuring, electromedical, and co...                116   
                   Scientific research and development services                      497   
                   Electronic components and products, n.e.c.                        747   
                   Aerospace products and parts manufacturing                         28   
                   Farm supplies merchant wholesalers                                  7   
                   Water, steam, air conditioning, and irrigation ...                 52   
                   Aircraft and parts manufacturing                                   73   
                   Footwear Manufacturing                                              5   
                   Paint, coating, and adhesives                                       5   
                   Securities, commodities, funds, trusts, and oth...                375   
                   Nonmetallic mineral mining and quarrying                            3   
                   Not specified utilities                                             3   
                   Other transportation equipment manufacturing                        3   
                   Wired telecommunications carriers                                  83   
                   Labor unions                                                       27   
                   Professional and commercial equipment and suppl...                 75   
                   Medical equipment and supplies                                    226   
                   Justice, public order, and safety activities                      319   
                   Support activities for mining                                      11   
                   Electronics stores                                                158   
                   Sewage treatment facilities                                        14   
                   Soun